# Code:

In [1]:
import os
import time
from datetime import datetime, timezone, timedelta
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)


In [2]:
# Kindly use your own API key
API_KEY = ""

CITY = "Karachi,PK"   

USE_COORDS = False
LAT = 31.5204
LON = 74.3587


HEAT_INDEX_ALERT_C = 40.0      
PLAIN_TEMP_ALERT_C  = 40.0      
RAIN_3H_ALERT_MM    = 50.0      
RAIN_24H_ALERT_MM   = 100.0     

# Logging
LOG_PATH = "weather_log.csv"
TZ_OFFSET_HOURS = 5  
LOCAL_TZ = timezone(timedelta(hours=TZ_OFFSET_HOURS))


PLOT_LAST_N = 200


In [3]:
def _owm_params():
    if USE_COORDS:
        return {"lat": LAT, "lon": LON, "appid": API_KEY, "units": "metric"}
    else:
        return {"q": CITY, "appid": API_KEY, "units": "metric"}

def fetch_current_weather():
    """Get current weather (temp, humidity, rain last 1h, wind)."""
    url = "https://api.openweathermap.org/data/2.5/weather"
    r = requests.get(url, params=_owm_params(), timeout=20)
    r.raise_for_status()
    data = r.json()

    temp = data["main"]["temp"]
    humidity = data["main"]["humidity"]
    wind_speed = data.get("wind", {}).get("speed", np.nan)
    rain_1h = data.get("rain", {}).get("1h", 0.0)  

    return {
        "temp": float(temp),
        "humidity": float(humidity),
        "wind_speed": float(wind_speed) if wind_speed is not None else np.nan,
        "rain_1h": float(rain_1h),
        "raw": data,
    }

def fetch_forecast_3h():
    """Get 5-day / 3-hourly forecast and compute rainfall for next 3h and 24h."""
    url = "https://api.openweathermap.org/data/2.5/forecast"
    r = requests.get(url, params=_owm_params(), timeout=20)
    r.raise_for_status()
    data = r.json()


    df = pd.json_normalize(data["list"])
    
    df["time_utc"] = pd.to_datetime(df["dt"], unit="s", utc=True)
    df["time_local"] = df["time_utc"].dt.tz_convert(LOCAL_TZ)
    df["rain_3h"] = df.get("rain.3h", pd.Series([0]*len(df))).fillna(0.0)

    
    rain_next_3h = float(df.iloc[0]["rain_3h"]) if len(df) > 0 else 0.0
    rain_next_24h = float(df.iloc[:8]["rain_3h"].sum()) if len(df) >= 1 else 0.0

    return {
        "rain_next_3h": rain_next_3h,
        "rain_next_24h": rain_next_24h,
        "df_forecast": df,
        "raw": data,
    }

def c_to_f(c):
    return (c * 9/5) + 32.0

def f_to_c(f):
    return (f - 32.0) * 5/9

def heat_index_celsius(temp_c, rh):
    """
    Compute Heat Index in °C using the Rothfusz regression (NOAA).
    Valid mainly for T > ~26.7°C and RH > 40%. We'll still compute for reference.
    """
    T = c_to_f(temp_c)
    R = rh
    
    HI_f = (-42.379 + 2.04901523*T + 10.14333127*R
            - 0.22475541*T*R - 6.83783e-3*T*T - 5.481717e-2*R*R
            + 1.22874e-3*T*T*R + 8.5282e-4*T*R*R - 1.99e-6*T*T*R*R)
    
    return f_to_c(HI_f)

def assess_risks(temp_c, rh, rain_1h, rain_next_3h, rain_next_24h):
    hi_c = heat_index_celsius(temp_c, rh)
    heat_alert = (hi_c >= HEAT_INDEX_ALERT_C) or (temp_c >= PLAIN_TEMP_ALERT_C)
    flood_alert = (rain_next_3h >= RAIN_3H_ALERT_MM) or (rain_next_24h >= RAIN_24H_ALERT_MM)

    reasons = []
    if heat_alert:
        if hi_c >= HEAT_INDEX_ALERT_C:
            reasons.append(f"Heat Index {hi_c:.1f}°C ≥ {HEAT_INDEX_ALERT_C}°C")
        if temp_c >= PLAIN_TEMP_ALERT_C:
            reasons.append(f"Temp {temp_c:.1f}°C ≥ {PLAIN_TEMP_ALERT_C}°C")
    if flood_alert:
        if rain_next_3h >= RAIN_3H_ALERT_MM:
            reasons.append(f"Rain next 3h {rain_next_3h:.1f} mm ≥ {RAIN_3H_ALERT_MM} mm")
        if rain_next_24h >= RAIN_24H_ALERT_MM:
            reasons.append(f"Rain next 24h {rain_next_24h:.1f} mm ≥ {RAIN_24H_ALERT_MM} mm")

    return {
        "heat_alert": bool(heat_alert),
        "flood_alert": bool(flood_alert),
        "heat_index_c": hi_c,
        "reasons": reasons,
    }

def now_local():
    return datetime.now(tz=LOCAL_TZ)

def log_row(row_dict, path=LOG_PATH):
    """Append a single row to CSV (create if missing)."""
    df_row = pd.DataFrame([row_dict])
    if not os.path.exists(path):
        df_row.to_csv(path, index=False)
    else:
        df_row.to_csv(path, mode="a", index=False, header=False)

def load_log(path=LOG_PATH):
    if not os.path.exists(path):
        return pd.DataFrame()
    df = pd.read_csv(path, parse_dates=["time_local"])
    return df

def plot_history(path=LOG_PATH, last_n=PLOT_LAST_N):
    df = load_log(path)
    if df.empty:
        print("No log data yet.")
        return

    df_tail = df.tail(last_n)

    # Plot Temperature
    plt.figure(figsize=(10, 4))
    plt.plot(df_tail["time_local"], df_tail["temp_c"], label="Temperature (°C)")
    plt.ylabel("°C")
    plt.xlabel("Time")
    plt.title("Temperature history")
    plt.legend()
    plt.tight_layout()
    plt.show()

    
    plt.figure(figsize=(10, 4))
    plt.plot(df_tail["time_local"], df_tail["rain_1h_mm"], label="Rain last 1h (mm)")
    plt.plot(df_tail["time_local"], df_tail["rain_next_3h_mm"], label="Rain next 3h (mm)")
    plt.plot(df_tail["time_local"], df_tail["rain_next_24h_mm"], label="Rain next 24h (mm)")
    plt.ylabel("mm")
    plt.xlabel("Time")
    plt.title("Rainfall history")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
if not API_KEY or API_KEY == "PASTE_YOUR_OPENWEATHERMAP_KEY_HERE":
    raise ValueError("Please paste your OpenWeatherMap API key in the API_KEY variable and run again.")


cur = fetch_current_weather()
fc  = fetch_forecast_3h()


risk = assess_risks(
    temp_c=cur["temp"],
    rh=cur["humidity"],
    rain_1h=cur["rain_1h"],
    rain_next_3h=fc["rain_next_3h"],
    rain_next_24h=fc["rain_next_24h"],
)


row = {
    "time_local": now_local(),
    "city": CITY if not USE_COORDS else f"{LAT},{LON}",
    "temp_c": cur["temp"],
    "humidity_pct": cur["humidity"],
    "wind_speed_ms": cur["wind_speed"],
    "rain_1h_mm": cur["rain_1h"],
    "rain_next_3h_mm": fc["rain_next_3h"],
    "rain_next_24h_mm": fc["rain_next_24h"],
    "heat_index_c": risk["heat_index_c"],
    "heat_alert": risk["heat_alert"],
    "flood_alert": risk["flood_alert"],
    "reasons": "; ".join(risk["reasons"]),
}


log_row(row, LOG_PATH)


print(f"[{row['time_local']}] — {row['city']}")
print(f"Temp: {row['temp_c']:.1f}°C | RH: {row['humidity_pct']:.0f}% | Wind: {row['wind_speed_ms']:.1f} m/s")
print(f"Rain: last 1h {row['rain_1h_mm']:.1f} mm | next 3h {row['rain_next_3h_mm']:.1f} mm | next 24h {row['rain_next_24h_mm']:.1f} mm")
print(f"Heat Index: {row['heat_index_c']:.1f}°C")

if row["heat_alert"] or row["flood_alert"]:
    print("⚠️ ALERTS:")
    if row["heat_alert"]:
        print("  • Heatwave risk")
    if row["flood_alert"]:
        print("  • Flood risk")
    if row["reasons"]:
        print("  Reasons:")
        for reason in risk["reasons"]:
            print(f"   - {reason}")
else:
    print("✅ No alerts triggered at this moment.")


plot_history(LOG_PATH, PLOT_LAST_N)


In [ ]:
INTERVAL_MIN = 30  

def run_once_and_log():
    cur = fetch_current_weather()
    fc  = fetch_forecast_3h()
    risk = assess_risks(
        temp_c=cur["temp"],
        rh=cur["humidity"],
        rain_1h=cur["rain_1h"],
        rain_next_3h=fc["rain_next_3h"],
        rain_next_24h=fc["rain_next_24h"],
    )
    row = {
        "time_local": now_local(),
        "city": CITY if not USE_COORDS else f"{LAT},{LON}",
        "temp_c": cur["temp"],
        "humidity_pct": cur["humidity"],
        "wind_speed_ms": cur["wind_speed"],
        "rain_1h_mm": cur["rain_1h"],
        "rain_next_3h_mm": fc["rain_next_3h"],
        "rain_next_24h_mm": fc["rain_next_24h"],
        "heat_index_c": risk["heat_index_c"],
        "heat_alert": risk["heat_alert"],
        "flood_alert": risk["flood_alert"],
        "reasons": "; ".join(risk["reasons"]),
    }
    log_row(row, LOG_PATH)
    print(f"[{row['time_local']}] Logged. Alerts: Heat={row['heat_alert']} Flood={row['flood_alert']}")

try:
    while True:
        run_once_and_log()
        plot_history(LOG_PATH, PLOT_LAST_N)
        time.sleep(INTERVAL_MIN * 60)
except KeyboardInterrupt:
    print("Monitoring stopped by user.")
